In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
NTHREAD = cpu_count()

import warnings
warnings.filterwarnings("ignore")

from IPython.display import display
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 50)

In [2]:
train  = pd.read_csv('../input/application_train.csv.zip')
train['is_train'] = 1

test  = pd.read_csv('../input/application_test.csv.zip')
test['is_train'] = 0

trte = pd.concat([train, test], ignore_index=True)

In [3]:
prev = pd.read_csv('../input/previous_application.csv.zip')
prev.sort_values('DAYS_DECISION', inplace=True)

In [4]:
user_id = pd.read_csv('../data/user_id_v8.csv.gz')

In [5]:
dup = user_id[user_id.duplicated('user_id', False)]
cnt = dup.groupby('user_id').size()
cnt.name = 'dup_cnt'
cnt = cnt.reset_index()
cnt['dup_id'] = range(cnt.shape[0])

dup = pd.merge(dup, cnt, on='user_id', how='left')
dup = pd.merge(dup, trte, on='SK_ID_CURR', how='left')
dup.sort_values(['user_id', 'DAYS_BIRTH'], ascending=[True, False], inplace=True)

dup['seq'] = 1
dup['seq'] = dup.groupby('user_id').seq.cumsum()-1

col = train.columns.tolist() + [c for c in dup.columns if c not in train.columns]
for c in ['dup_id', 'dup_cnt', 'user_id']:
    col.remove(c)
col = ['user_id'] + col

dup = dup[col].reset_index(drop=True)
dup

,user_id,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,...,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,is_train,seq
0,18.0,100279,0.0,Cash loans,M,N,Y,0,180000.0,254700.0,16407.0,225000.0,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,0.072508,-23946,365243,-6935.0,-3948,NaN,1,0,0,1,0,0,NaN,1.0,1,1,WEDNESDAY,16,0,0,0,0,0,0,XNA,NaN,0.458685,0.528093,0.3041,0.1854,0.9791,0.7144,...,0.7182,0.0000,0.32,0.2759,0.3333,0.0417,0.0000,0.2454,0.2861,0.0311,0.0092,reg oper account,block of flats,0.2230,Panel,No,0.0,0.0,0.0,0.0,-975.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,5.0,1,0
1,18.0,100124,0.0,Cash loans,M,N,Y,0,225000.0,448272.0,28647.0,396000.0,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,0.072508,-24051,365243,-7040.0,-4053,NaN,1,0,0,1,1,1,NaN,1.0,1,1,WEDNESDAY,17,0,0,0,0,0,0,XNA,NaN,0.710318,0.554947,0.2959,0.1785,0.9786,0.7076,...,0.7115,0.0000,0.32,0.2759,0.3333,0.3750,0.0000,0.2454,0.2741,0.0000,0.0062,reg oper account,block of flats,0.2131,Panel,No,0.0,0.0,0.0,0.0,-1080.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,8.0,1,1
2,38.0,100955,0.0,Cash loans,F,N,Y,0,112500.0,101880.0,10053.0,90000.0,Unaccompanied,Pensioner,Lower secondary,Single / not married,House / apartment,0.030755,-22321,365243,-13611.0,-2523,NaN,1,0,0,1,0,0,NaN,1.0,2,2,FRIDAY,10,0,0,0,0,0,0,XNA,NaN,0.664900,0.504681,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,1.0,-1012.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,3.0,1,0
3,38.0,190916,NaN,Cash loans,F,N,Y,0,135000.0,238500.0,24565.5,238500.0,Unaccompanied,Pensioner,Lower secondary,Single / not married,House / apartment,0.030755,-22609,365243,-73.0,-2811,NaN,1,0,0,1,1,0,NaN,1.0,2,2,SATURDAY,10,0,0,0,0,0,0,XNA,0.279026,0.504692,0.466864,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,-1300.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0,0,1
4,40.0,178701,0.0,Revolving loans,F,N,Y,0,90000.0,270000.0,13500.0,270000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-16416,-1223,-2542.0,-4382,NaN,1,1,0,1,0,0,NaN,2.0,2,2,WEDNESDAY,12,0,0,0,0,1,1,Business Entity Type 3,0.479804,0.514119,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [6]:
dup[dup.SK_ID_CURR==157771]

,user_id,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,...,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,is_train,seq
665,3138.0,157771,0.0,Cash loans,F,N,Y,0,180000.0,297000.0,9396.0,297000.0,Unaccompanied,State servant,Higher education,Married,House / apartment,0.04622,-12435,-704,-1377.0,-2862,NaN,1,1,1,1,1,0,Core staff,2.0,1,1,WEDNESDAY,16,0,0,0,0,0,0,School,0.41129,0.687431,0.171468,0.1031,0.1115,0.9786,NaN,...,NaN,NaN,0.0,0.2069,0.1667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,block of flats,0.0702,"Stone, brick",No,0.0,0.0,0.0,0.0,-520.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,3.0,1,0


In [7]:
dup[dup.SK_ID_CURR==100066]

,user_id,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,...,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,is_train,seq
666,3138.0,100066,NaN,Cash loans,F,N,Y,0,315000.0,364896.0,28957.5,315000.0,Unaccompanied,State servant,Higher education,Married,House / apartment,0.04622,-12744,-1013,-1686.0,-3171,NaN,1,1,0,1,0,0,Core staff,2.0,1,1,THURSDAY,18,0,0,0,0,0,0,School,0.718507,0.808788,0.522697,0.1031,0.1115,0.9781,NaN,...,NaN,NaN,0.0,0.2069,0.1667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,block of flats,0.0702,"Stone, brick",No,0.0,0.0,0.0,0.0,-829.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0,0,1


In [8]:
prev[prev.SK_ID_CURR==157771]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
115456,1295424,157771,Consumer loans,7589.925,68791.5,76054.5,0.0,68791.5,MONDAY,15,Y,1,0.0,NaN,NaN,XAP,Approved,-520,Cash through the bank,XAP,NaN,New,Mobile,POS,XNA,Country-wide,15,Connectivity,12.0,middle,POS mobile without interest,365243.0,-489.0,-159.0,-159.0,-157.0,0.0
1265969,1354392,157771,Consumer loans,10729.440,100152.0,110727.0,0.0,100152.0,SATURDAY,14,Y,1,0.0,NaN,NaN,XAP,Approved,-340,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,43,Connectivity,12.0,low_normal,POS mobile without interest,365243.0,-309.0,21.0,-9.0,-3.0,0.0
1260816,2631328,157771,Consumer loans,12382.650,92110.5,101839.5,0.0,92110.5,TUESDAY,13,Y,1,0.0,NaN,NaN,XAP,Approved,-141,Cash through the bank,XAP,Unaccompanied,Repeater,Computers,POS,XNA,Country-wide,10,Connectivity,12.0,high,POS mobile with interest,365243.0,-111.0,219.0,365243.0,365243.0,1.0
313895,2793303,157771,Revolving loans,2250.000,45000.0,45000.0,NaN,45000.0,TUESDAY,13,Y,1,NaN,NaN,NaN,XAP,Approved,-141,XNA,XAP,Unaccompanied,Repeater,XNA,Cards,walk-in,Country-wide,10,Connectivity,0.0,XNA,Card Street,365243.0,365243.0,365243.0,365243.0,365243.0,0.0
542458,2664704,157771,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,9,Y,1,NaN,NaN,NaN,XNA,Canceled,-15,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
prev[prev.SK_ID_CURR==100066]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
499369,1008517,100066,Consumer loans,7589.925,68791.5,76054.5,0.0,68791.5,MONDAY,15,Y,1,0.0,NaN,NaN,XAP,Approved,-829,Cash through the bank,XAP,NaN,New,Mobile,POS,XNA,Country-wide,15,Connectivity,12.0,middle,POS mobile without interest,365243.0,-798.0,-468.0,-468.0,-466.0,0.0
1643672,2551852,100066,Consumer loans,10729.440,100152.0,110727.0,0.0,100152.0,SATURDAY,14,Y,1,0.0,NaN,NaN,XAP,Approved,-649,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,43,Connectivity,12.0,low_normal,POS mobile without interest,365243.0,-618.0,-288.0,-318.0,-312.0,0.0
692281,1852787,100066,Revolving loans,2250.000,45000.0,45000.0,NaN,45000.0,TUESDAY,13,Y,1,NaN,NaN,NaN,XAP,Approved,-450,XNA,XAP,Unaccompanied,Repeater,XNA,Cards,walk-in,Country-wide,10,Connectivity,0.0,XNA,Card Street,365243.0,365243.0,365243.0,365243.0,365243.0,0.0
616339,1084766,100066,Consumer loans,12382.650,92110.5,101839.5,0.0,92110.5,TUESDAY,13,Y,1,0.0,NaN,NaN,XAP,Approved,-450,Cash through the bank,XAP,Unaccompanied,Repeater,Computers,POS,XNA,Country-wide,10,Connectivity,12.0,high,POS mobile with interest,365243.0,-420.0,-90.0,-90.0,-88.0,1.0
988234,2530269,100066,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,9,Y,1,NaN,NaN,NaN,XNA,Canceled,-324,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
1387179,2389193,100066,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,12,Y,1,NaN,NaN,NaN,XNA,Canceled,-310,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
465133,2827980,100066,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,13,Y,1,NaN,NaN,NaN,XNA,Canceled,-310,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
292816,2081566,100066,Cash loans,17244.765,450000.0,545040.0,NaN,450000.0,TUESDAY,13,Y,1,NaN,NaN,NaN,Other,Refused,-310,Cash through the bank,HC,Unaccompanied,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,48.0,low_normal,Cash Street: low,NaN,NaN,NaN,NaN,NaN,NaN
1122719,1941379,100066,Cash loans,9400.050,297000.0,297000.0,NaN,297000.0,WEDNESDAY,16,Y,1,NaN,NaN,NaN,Payments on other loans,Approved,-309,Cash through the bank,XAP,Unaccompanied,Repeater,XNA,Cash,walk-in,Contact center,-1,XNA,45.0,low_action,Cash Street: low,365243.0,-279.0,1041.0,365243.0,365243.0,0.0


In [10]:
col_num = prev.select_dtypes(exclude='O').columns.tolist()
col_cat = prev.select_dtypes('O').columns.tolist()

In [11]:
date_feature = ['DAYS_DECISION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
               'DAYS_LAST_DUE', 'DAYS_TERMINATION']

keys = ['NAME_CONTRACT_TYPE', 'AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 
        'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START', 
        'HOUR_APPR_PROCESS_START', 'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
        'RATE_DOWN_PAYMENT', 'NAME_CONTRACT_STATUS', 'NAME_TYPE_SUITE', 'PRODUCT_COMBINATION']

In [12]:
for c in date_feature:
    col_num.remove(c)

In [13]:
prev[col_num] = prev[col_num].fillna(-1) # awful 
prev[col_cat] = prev[col_cat].fillna('-NA-')

In [14]:
prev[prev.SK_ID_CURR==157771]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
115456,1295424,157771,Consumer loans,7589.925,68791.5,76054.5,0.0,68791.5,MONDAY,15,Y,1,0.0,-1.0,-1.0,XAP,Approved,-520,Cash through the bank,XAP,-NA-,New,Mobile,POS,XNA,Country-wide,15,Connectivity,12.0,middle,POS mobile without interest,365243.0,-489.0,-159.0,-159.0,-157.0,0.0
1265969,1354392,157771,Consumer loans,10729.440,100152.0,110727.0,0.0,100152.0,SATURDAY,14,Y,1,0.0,-1.0,-1.0,XAP,Approved,-340,Cash through the bank,XAP,-NA-,Repeater,Mobile,POS,XNA,Country-wide,43,Connectivity,12.0,low_normal,POS mobile without interest,365243.0,-309.0,21.0,-9.0,-3.0,0.0
1260816,2631328,157771,Consumer loans,12382.650,92110.5,101839.5,0.0,92110.5,TUESDAY,13,Y,1,0.0,-1.0,-1.0,XAP,Approved,-141,Cash through the bank,XAP,Unaccompanied,Repeater,Computers,POS,XNA,Country-wide,10,Connectivity,12.0,high,POS mobile with interest,365243.0,-111.0,219.0,365243.0,365243.0,1.0
313895,2793303,157771,Revolving loans,2250.000,45000.0,45000.0,-1.0,45000.0,TUESDAY,13,Y,1,-1.0,-1.0,-1.0,XAP,Approved,-141,XNA,XAP,Unaccompanied,Repeater,XNA,Cards,walk-in,Country-wide,10,Connectivity,0.0,XNA,Card Street,365243.0,365243.0,365243.0,365243.0,365243.0,0.0
542458,2664704,157771,Cash loans,-1.000,0.0,0.0,-1.0,-1.0,TUESDAY,9,Y,1,-1.0,-1.0,-1.0,XNA,Canceled,-15,XNA,XAP,-NA-,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,-1.0,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0


# Backwards

In [15]:
def backwards(args):
    start_ix, last_ix = args
    li = []
    curr_id_bk, seq_bk, day_bk = None, None, None
    for i in range(start_ix, last_ix):
        curr_id, seq, day = dup.iloc[i, 1], dup.iloc[i, -1], dup.iloc[i, 18] 
        if seq ==0:
            curr_id_bk, seq_bk, day_bk = curr_id, seq, day
            continue
        else:
            day_diff = day - day_bk
            
            if day_diff==0:
                curr_id_bk, seq_bk, day_bk = curr_id, seq, day
                continue
            
            prev_bk = prev[prev.SK_ID_CURR==curr_id_bk]
            
            prev_cur = prev[prev.SK_ID_CURR==curr_id]
            prev_cur['SK_ID_CURR'] = curr_id_bk
            for c in date_feature:
                prev_cur.loc[prev[c]!=365243, c] -= day_diff
            
            # ==== future ====
            a = prev_bk.groupby(keys).size()
            a.name = 'a'
            b = prev_cur.groupby(keys).size()
            b.name = 'b'
            
            tmp = pd.concat([a, b], axis=1, join='outer')
            tmp[['a', 'b']] = tmp[['a', 'b']].fillna(0)
            tmp['need'] = tmp['a'] - tmp['b']
            
            base = tmp[tmp.need < 0][['need']]
            base.need = base.need.map(abs)
            
            aug = pd.DataFrame()
            while base.shape[0] > 0:
                aug = aug.append(base.drop('need', axis=1))
                
                base.need -= 1
                base = base[base.need>0]
            
            if aug.shape[0]==0:
                df = pd.DataFrame()
            else:
                df = pd.merge(aug.reset_index(), prev_cur, on=keys, how='left' )
            # ==== future end ====
            
            
            li.append(df)
        
        curr_id_bk, seq_bk, day_bk = curr_id, seq, day
        
    return li

In [16]:
pd.concat(backwards((4, 6)))

,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,NAME_CONTRACT_STATUS,NAME_TYPE_SUITE,PRODUCT_COMBINATION,SK_ID_PREV,SK_ID_CURR,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,Revolving loans,-1.0,0.0,0.0,-1.0,-1.0,WEDNESDAY,11,Y,1,-1.0,Canceled,-NA-,Card Street,1851565,178701,-1.0,-1.0,XAP,0,XNA,XAP,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,-1.0,XNA,NaN,NaN,NaN,NaN,NaN,-1.0
1,Revolving loans,13500.0,270000.0,270000.0,-1.0,270000.0,WEDNESDAY,12,Y,1,-1.0,Approved,Unaccompanied,Card X-Sell,1536648,178701,-1.0,-1.0,XAP,0,XNA,XAP,Repeater,XNA,Cards,x-sell,Credit and cash offices,-1,XNA,0.0,XNA,0.0,49.0,365243.0,365243.0,365243.0,0.0


In [17]:
argmin = dup.groupby('user_id').seq.apply(np.argmin)
argmax = dup.groupby('user_id').seq.apply(np.argmax)
argmax +=1

argss = list(zip(argmin.values, argmax.values ))

In [18]:
pool = Pool(NTHREAD)
callback = pool.map(backwards, argss)
pool.close()

In [19]:
callback2 = sum( callback , [])

In [20]:
future_app = pd.concat(callback2)

In [21]:
future_app

,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,NAME_CONTRACT_STATUS,NAME_TYPE_SUITE,PRODUCT_COMBINATION,SK_ID_PREV,SK_ID_CURR,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,Cash loans,-1.000,0.0,0.0,-1.0,-1.0,WEDNESDAY,16,Y,1,-1.000000,Canceled,-NA-,Cash,1781858,100279,-1.0,-1.0,XNA,0,XNA,XAP,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,-1.0,XNA,NaN,NaN,NaN,NaN,NaN,-1.0
1,Cash loans,16408.395,225000.0,254700.0,-1.0,225000.0,WEDNESDAY,16,Y,1,-1.000000,Approved,Unaccompanied,Cash X-Sell: middle,1926820,100279,-1.0,-1.0,XNA,0,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,24.0,middle,365243.0,30.0,720.0,365243.0,365243.0,1.0
0,Cash loans,-1.000,0.0,0.0,-1.0,-1.0,FRIDAY,10,Y,1,-1.000000,Canceled,-NA-,Cash,1924661,100955,-1.0,-1.0,XNA,0,XNA,XAP,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,-1.0,XNA,NaN,NaN,NaN,NaN,NaN,-1.0
1,Cash loans,-1.000,0.0,0.0,-1.0,-1.0,THURSDAY,16,Y,1,-1.000000,Canceled,-NA-,Cash,1671021,100955,-1.0,-1.0,XNA,6,XNA,XAP,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,-1.0,XNA,NaN,NaN,NaN,NaN,NaN,-1.0
2,Cash loans,10055.655,90000.0,101880.0,-1.0,90000.0,FRIDAY,10,Y,1,-1.000000,Approved,Unaccompanied,Cash X-Sell: low,1870308,100955,-1.0,-1.0,XNA,0,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,low_normal,365243.0,30.0,360.0,365243.0,365243.0,1.0
3,Cash loans,22826.295,337500.0,408780.0,-1.0,337500.0,THURSDAY,16,Y,1,-1.000000,Refused,Unaccompanied,Cash X-Sell: low,1366077,100955,-1.0,-1.0,XNA,6,Cash through the bank,HC,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,24.0,low_normal,NaN,NaN,NaN,NaN,NaN,-1.0
0,Revolving loans,-1.000,0.0,0.0,-1.0,-1.0,WEDNESDAY,11,Y,1,-1.000000,Canceled,-NA-,Card Street,1851565,178701,-1.0,-1.0,XAP,0,XNA,XAP,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,-1.0,XNA,NaN,NaN,NaN,NaN,NaN,-1.0
1,Revolving loans,13500.000,270000.0,270000.0,-1.0,270000.0,WEDNESDAY,12,Y,1,-1.000000,Approved,Unaccompanied,Card X-Sell,1536648,178701,-1.0,-1.0,XAP,0,XNA,XAP,Repeater,XNA,Cards,x-sell,Credit and cash offices,-1,XNA,0.0,XNA,0.0,49.0,365243.0,365243.0,365243.0,0.0
0,Cash loans,-1.000,0.0,0.0,-1.0,-1.0,THURSDAY,14,Y,1,-1.000000,Canceled,-NA-,Cash,1324640,247772,-1.0,-1.0,XNA,0,XNA,XAP,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,-1.0,XNA,NaN,NaN,NaN,NaN,NaN,-1.0
1,Cash loans,37263.195,630000.0,729792.0,-1.0,630000.0,THURSDAY,14,Y,1,-1.000000,Approved,Unaccompanied,Cash X-Sell: middle,1937120,247772,-1.0,-1.0,XNA,0,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,36.0,middle,365243.0,30.0,1080.0,365243.0,365243.0,1.0


# check

In [22]:
future_app.SK_ID_CURR.unique().shape

(8154,)

In [23]:
future_app.SK_ID_PREV.unique().shape

(24948,)

# merge with clean prev

In [24]:
prev = pd.read_csv('../input/previous_application.csv.zip')
col = prev.columns.tolist()
prev.drop(['SK_ID_CURR']+date_feature, axis=1, inplace=True)

In [25]:
future_app_base = future_app[['SK_ID_PREV', 'SK_ID_CURR']+date_feature]

In [26]:
future_app.shape

(30156, 37)

In [27]:
prev.shape

(1670214, 30)

In [28]:
future_app2 = pd.merge(future_app_base, prev, on='SK_ID_PREV', how='left')[col]

In [29]:
future_app2.DAYS_DECISION.describe() #[['SK_ID_PREV', 'SK_ID_CURR']+date_feature]

count    30156.000000
mean        32.820931
std        262.864317
min      -2572.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       8687.000000
Name: DAYS_DECISION, dtype: float64

In [30]:
prev2 = future_app2[future_app2.DAYS_DECISION<0]
future_app3 = future_app2[future_app2.DAYS_DECISION>=0]
prev2.shape, future_app3.shape

((2480, 37), (27676, 37))

# output

In [31]:
future_app3.to_csv('../data/future_application_v3.csv.gz', index=False, compression='gzip')

In [32]:
prev2.to_csv('../data/prev_backwards_v3.csv.gz', index=False, compression='gzip')

# concat prev

In [33]:
prev_forwards = pd.read_csv('../data/prev_forwards_v3.csv.gz')

In [34]:
prev2.shape, prev_forwards.shape

((2480, 37), (1671673, 37))

In [35]:
pd.merge(prev_forwards, prev2, on=['SK_ID_PREV', 'SK_ID_CURR'], how='inner')

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE_x,AMT_ANNUITY_x,AMT_APPLICATION_x,AMT_CREDIT_x,AMT_DOWN_PAYMENT_x,AMT_GOODS_PRICE_x,WEEKDAY_APPR_PROCESS_START_x,HOUR_APPR_PROCESS_START_x,FLAG_LAST_APPL_PER_CONTRACT_x,NFLAG_LAST_APPL_IN_DAY_x,RATE_DOWN_PAYMENT_x,RATE_INTEREST_PRIMARY_x,RATE_INTEREST_PRIVILEGED_x,NAME_CASH_LOAN_PURPOSE_x,NAME_CONTRACT_STATUS_x,DAYS_DECISION_x,NAME_PAYMENT_TYPE_x,CODE_REJECT_REASON_x,NAME_TYPE_SUITE_x,NAME_CLIENT_TYPE_x,NAME_GOODS_CATEGORY_x,NAME_PORTFOLIO_x,NAME_PRODUCT_TYPE_x,CHANNEL_TYPE_x,SELLERPLACE_AREA_x,NAME_SELLER_INDUSTRY_x,CNT_PAYMENT_x,NAME_YIELD_GROUP_x,PRODUCT_COMBINATION_x,DAYS_FIRST_DRAWING_x,DAYS_FIRST_DUE_x,DAYS_LAST_DUE_1ST_VERSION_x,DAYS_LAST_DUE_x,DAYS_TERMINATION_x,NFLAG_INSURED_ON_APPROVAL_x,NAME_CONTRACT_TYPE_y,AMT_ANNUITY_y,AMT_APPLICATION_y,AMT_CREDIT_y,AMT_DOWN_PAYMENT_y,AMT_GOODS_PRICE_y,WEEKDAY_APPR_PROCESS_START_y,HOUR_APPR_PROCESS_START_y,FLAG_LAST_APPL_PER_CONTRACT_y,NFLAG_LAST_APPL_IN_DAY_y,RATE_DOWN_PAYMENT_y,RATE_INTEREST_PRIMARY_y,RATE_INTEREST_PRIVILEGED_y,NAME_CASH_LOAN_PURPOSE_y,NAME_CONTRACT_STATUS_y,DAYS_DECISION_y,NAME_PAYMENT_TYPE_y,CODE_REJECT_REASON_y,NAME_TYPE_SUITE_y,NAME_CLIENT_TYPE_y,NAME_GOODS_CATEGORY_y,NAME_PORTFOLIO_y,NAME_PRODUCT_TYPE_y,CHANNEL_TYPE_y,SELLERPLACE_AREA_y,NAME_SELLER_INDUSTRY_y,CNT_PAYMENT_y,NAME_YIELD_GROUP_y,PRODUCT_COMBINATION_y,DAYS_FIRST_DRAWING_y,DAYS_FIRST_DUE_y,DAYS_LAST_DUE_1ST_VERSION_y,DAYS_LAST_DUE_y,DAYS_TERMINATION_y,NFLAG_INSURED_ON_APPROVAL_y


In [36]:
df = pd.concat([prev_forwards, prev2]).sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop=True)

In [37]:
df.shape

(1674153, 37)

In [38]:
df.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,1369693,100001,Consumer loans,3951.000,24835.5,23787.0,2520.0,24835.5,FRIDAY,13,Y,1,0.104326,NaN,NaN,XAP,Approved,-1740,Cash through the bank,XAP,Family,Refreshed,Mobile,POS,XNA,Country-wide,23,Connectivity,8.0,high,POS mobile with interest,365243.0,-1709.0,-1499.0,-1619.0,-1612.0,0.0
1,1038818,100002,Consumer loans,9251.775,179055.0,179055.0,0.0,179055.0,SATURDAY,9,Y,1,0.000000,NaN,NaN,XAP,Approved,-606,XNA,XAP,NaN,New,Vehicles,POS,XNA,Stone,500,Auto technology,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0
2,1810518,100003,Cash loans,98356.995,900000.0,1035882.0,NaN,900000.0,FRIDAY,12,Y,1,NaN,NaN,NaN,XNA,Approved,-746,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
3,2396755,100003,Consumer loans,6737.310,68809.5,68053.5,6885.0,68809.5,SATURDAY,15,Y,1,0.100061,NaN,NaN,XAP,Approved,-2341,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,Country-wide,200,Consumer electronics,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0
4,2636178,100003,Consumer loans,64567.665,337500.0,348637.5,0.0,337500.0,SUNDAY,17,Y,1,0.000000,NaN,NaN,XAP,Approved,-828,Cash through the bank,XAP,Family,Refreshed,Furniture,POS,XNA,Stone,1400,Furniture,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0


In [39]:
df.to_csv('../data/prev_new_v4.csv.gz', index=False, compression='gzip')